# Section 5

In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

Implement the mean absolute error:
$$
MAE = \frac{1}{N}\sum_{i=1}^N |y_i-x_i^\top\theta|
$$

In [2]:
def get_MAE(theta, X, y):
    # --------------
    # Your Code Here
    # --------------
    mae = np.mean(np.abs(y - X @ theta))
    return mae

## Question 7.2
Implement the problem from Question 7.1 (Use the `GLPK` solver)

In [3]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X, X_test, Y, Y_test = train_test_split(diabetes['data'], 
                                        np.expand_dims(diabetes['target'], 1), 
                                        test_size=0.5, random_state=0)

# add the bias term
X = np.hstack([np.ones([X.shape[0],1]), X])
X_test = np.hstack([np.ones([X_test.shape[0],1]), X_test])

print(X.shape)
print(Y.shape)

(221, 11)
(221, 1)


### Robust Cross Validation

In [4]:
# get dimensions
N = X.shape[0]
k = np.floor(0.75*N).astype('int')
D = X.shape[1]

print('N shape: \t{}'.format(N))
print('k shape: \t{}'.format(k))
print('D shape: \t{}'.format(D))

N shape: 	221
k shape: 	165
D shape: 	11


In [5]:
# define coefficients 
b = np.vstack((np.ones((N,1)), k))
A = np.vstack((np.identity(N), np.ones((1,N))))

# define the decision variables
theta = cp.Variable((D,1))
p = cp.Variable((N+1,1)) # beta = p[:-1], alpha = p[-1]
v = cp.Variable((D,1))

# define contraints
constraints =   [   A.T @ p >= 1/k * (Y - X @ theta),
                    A.T @ p >= -1/k * (Y - X @ theta),
                    theta <= v,
                    -theta <= v,
                    p[:-1] >= 0
                ]

In [19]:
# lambdas
lambdas = np.logspace(-5.0, -1.0, num=50)
best_mae = np.inf # initial value for best_mae

for lam in lambdas:
    ###
    objective = cp.Minimize( b.T @ p + lam * np.ones((D,1)).T @ v )
    prob = cp.Problem(objective, constraints)
    cost = prob.solve(solver=cp.GLPK)
    ###

    # get z by sorting the absolute error of each sample
    ind = np.argsort( np.abs(Y - X @ theta.value), axis=0)[::-1]
    z = np.zeros(N)
    z[ind[:k]] = 1
    z = z.astype(bool)

    X_train = X[z]
    X_val = X[~z]
    Y_train = Y[z]
    Y_val = Y[~z]

    # Check if the sorted training indices get the same cost as the solver
    # print(get_MAE(theta.value, X_train, Y_train) + lam * np.linalg.norm(theta.value, 1))
    # print(cost)

    mae = get_MAE(theta.value, X_val, Y_val)
    if mae < best_mae:
        best_z = z
        best_mae = mae
        best_theta = theta.value
        best_lambda = lam

    # print("Lambda: {:.12f}".format(lam))
    # print(color.BOLD + 'Validation Results' + color.END)
    # print('MAE: {:.12f}'.format(get_MAE(theta.value, X_val, Y_val)))
    # print(color.BOLD + 'Training Results' + color.END)
    # print('MAE: {:.12f}'.format(get_MAE(theta.value, X_train, Y_train)))
    # print(color.BOLD + 'Test Results' + color.END)
    # print('MAE: {:.12f}'.format(get_MAE(theta.value, X_test, Y_test)))
    # print(50*'-')

print("Best lambda: {:.12f}".format(best_lambda))
print(color.BOLD + 'Validation Results' + color.END)
print('MAE: {:.12f}'.format(get_MAE(best_theta, X[~best_z], Y[~best_z])))
print(color.BOLD + 'Training Results' + color.END)
print('MAE: {:.12f}'.format(get_MAE(best_theta, X[best_z], Y[best_z])))
print(color.BOLD + 'Test Results' + color.END)
print('MAE: {:.12f}'.format(get_MAE(best_theta, X_test, Y_test)))
print(50*'-')

Best lambda: 0.003393221772
Validation Results
MAE: 8.886451587972
Training Results
MAE: 54.206625371223
Test Results
MAE: 44.973977912733
--------------------------------------------------


In [20]:
# print best values
print('Best indices for training set: \n{}'.format(best_z.astype(int).T))
print('Cardinality of best training set: \n{}'.format(best_z.sum()))
print('Best theta: \n{}'.format(best_theta))
print('Best lambda: \t{:.12f}'.format(best_lambda))
print('Best MAE val:\t{:.12f}'.format(best_mae))

Best indices for training set: 
[1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1
 1 1 0 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 0
 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0
 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1]
Cardinality of best training set: 
165
Best theta: 
[[ 1.51471814e+02]
 [-2.30054050e+01]
 [-2.15390319e+02]
 [ 4.58065856e+02]
 [ 2.51256798e+02]
 [ 0.00000000e+00]
 [-4.37811490e+01]
 [-2.26305403e+02]
 [ 1.44068900e-13]
 [ 6.66269812e+02]
 [ 2.20606674e-14]]
Best lambda: 	0.003393221772
Best MAE val:	8.886451587972


### Standard Cross Validation

In [21]:
# split the X, Y into  X_train, Y_train and X_val, Y_val
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=42)

print(X_train.shape)
print(Y_train.shape)

(165, 11)
(165, 1)


In [22]:
# get dimensions for training
N = X_train.shape[0]
D = X_train.shape[1]

In [24]:
# define the decision variables
u = cp.Variable((N,1))
v = cp.Variable((D,1))
theta = cp.Variable((D,1))

# lambdas
lambdas = np.logspace(-5.0, -1.0, num=50)
best_mae = np.inf # initial value for best_mae

for lam in lambdas:
    ###
    objective = cp.Minimize(1/N * np.ones((N,1)).T @ u + lam * np.ones((D,1)).T @ v)
    constraints = [ Y_train - X_train @ theta <= u,
                -Y_train + X_train @ theta <= u,
                theta <= v,
                -theta <= v,
    ]

    prob = cp.Problem(objective, constraints)
    cost = prob.solve(solver=cp.GLPK)

    ####   
    mae = get_MAE(theta.value, X_val, Y_val)
    if mae < best_mae:
        best_mae = mae
        best_theta = theta.value
        best_lambda = lam

    # print("Lambda: {:.12f}".format(lam))
    # print(color.BOLD + 'Validation Results' + color.END)
    # print('MAE: {:.12f}'.format(get_MAE(theta.value, X_val, Y_val)))
    # print(color.BOLD + 'Training Results' + color.END)
    # print('MAE: {:.12f}'.format(get_MAE(theta.value, X_train, Y_train)))
    # print(color.BOLD + 'Test Results' + color.END)
    # print('MAE: {:.12f}'.format(get_MAE(theta.value, X_test, Y_test)))
    # print(50*'-')

print("Best lambda: {:.12f}".format(best_lambda))
print(color.BOLD + 'Validation Results' + color.END)
print('MAE: {:.12f}'.format(get_MAE(best_theta, X_val, Y_val)))
print(color.BOLD + 'Training Results' + color.END)
print('MAE: {:.12f}'.format(get_MAE(best_theta, X_train, Y_train)))
print(color.BOLD + 'Test Results' + color.END)
print('MAE: {:.12f}'.format(get_MAE(best_theta, X_test, Y_test)))
print(50*'-')

Best lambda: 0.007196856730
Validation Results
MAE: 44.988498062858
Training Results
MAE: 45.741476031390
Test Results
MAE: 46.574314961565
--------------------------------------------------


In [25]:
# print best values
print('Best theta: \n{}'.format(best_theta))
print('Best lambda: \t{:.12f}'.format(best_lambda))
print('Best MAE val:\t{:.12f}'.format(best_mae))

Best theta: 
[[ 141.60326812]
 [   0.        ]
 [-115.76842328]
 [ 300.86042909]
 [ 114.79739473]
 [   0.        ]
 [   0.        ]
 [-166.19218658]
 [   0.        ]
 [ 625.95381683]
 [   0.        ]]
Best lambda: 	0.007196856730
Best MAE val:	44.988498062858
